In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', 40)

In [3]:
url = "https://www.data.gouv.fr/fr/pages/donnees-des-elections/"

In [4]:
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [5]:
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
soup

<!DOCTYPE html>
<html data-fr-scheme="light" data-fr-theme="light" lang="fr">
<head>
<meta charset="utf-8"/>
<title>Les données relatives aux élections - data.gouv.fr</title>
<meta content="website" property="og:type"/>
<meta content="Les données relatives aux élections - data.gouv.fr" property="og:title"/>
<link href="http://www.etalab.gouv.fr/" rel="author"/>
<meta content="Page de référencement des jeux de données relatifs aux élections sur data.gouv.fr" name="description"/>
<meta content="Page de référencement des jeux de données relatifs aux élections sur data.gouv.fr" property="og:description"/>
<meta content="https://static.data.gouv.fr/_themes/gouvfr/img/logo-social.png?_=1.2.4" property="og:image"/>
<meta content="opendata, udata, élections, électorales, municipales, présidentielles, européennes, législatives" name="keywords"/>
<meta content="https://static.data.gouv.fr/_themes/gouvfr/" name="theme-static-root"/>
<meta content="https://static.data.gouv.fr/static/" name="static

In [7]:
soup.select("h2")

[<h2>Élections Municipales</h2>,
 <h2>Élections Européennes</h2>,
 <h2>Élections législatives</h2>,
 <h2>Élections présidentielles</h2>,
 <h2>Élections Sénatoriales</h2>,
 <h2>Élections régionales</h2>,
 <h2>Élections Départementales</h2>,
 <h2>Élections métropolitaines</h2>,
 <h2>Élections cantonales</h2>,
 <h2>Référendums et  Consultations</h2>,
 <h2>Ressources/ Documentations</h2>,
 <h2>Jeux de données <sup>18</sup></h2>,
 <h2>Réutilisations <sup>6</sup></h2>,
 <h2 class="fs-xxl">Actions</h2>,
 <h2 class="h1 mb-md">Participez</h2>]

In [8]:
soup.select("ul:nth-child(39) > li > ul > li > a")

[<a href="https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour-par-bureaux-de-vote/20170511-093541/PR17_BVot_T2_FE.txt">par bureaux de vote</a>,
 <a href="https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour-par-communes/20170511-093054/Presidentielle_2017_Resultats_Communes_Tour_2_c.xls">par communes</a>,
 <a href="https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour/20170511-092258/Presidentielle_2017_Resultats_Tour_2_c.xls">par régions, départements, circonscriptions législatives et cantons</a>,
 <a href="https://www.data.gouv.fr/s/resources/elections-presidentielles-2017-2nd-tour-vote-des-francais-etablis-hors-de-france/20170509-121225/Presidentielle2017-FAE-2nd-tour-pays.csv">Résultats 2nd tour par pays</a>,
 <a href="https://www.data.gouv.fr/s/resources/elections-presidentielles-

In [9]:
url_list=[]
for element in soup.select("ul:nth-child(39) > li > ul > li > a"):
    if 'bureaux' in element["href"]:
        url_list.append(element["href"])

for element in soup.select("ul:nth-child(40) > li > ul > li > a"):
    if 'bureaux' in element["href"]:
        url_list.append(element["href"])
    
url_list

['https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour-par-bureaux-de-vote/20170511-093541/PR17_BVot_T2_FE.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2012-resultats-par-bureaux-de-vote-1/20150925-102751/PR12_Bvot_T1T2.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2007-resultats-par-bureaux-de-vote/20151001-154056/PR07_Bvot_T1T2.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2002-resultats-par-bureaux-de-vote/20150925-110503/PR02_BVot_T1T2.txt']

In [10]:
# One of the url is "hidden" behind a differen <p> tag. I need to add it manually to the list
url_list.append(soup.select("#app > main > section > div > div.container.markdown.py-lg > div > div > ul:nth-child(39) > li > ul > li:nth-child(7) > p > a")[0]["href"])
url_list

['https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour-par-bureaux-de-vote/20170511-093541/PR17_BVot_T2_FE.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2012-resultats-par-bureaux-de-vote-1/20150925-102751/PR12_Bvot_T1T2.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2007-resultats-par-bureaux-de-vote/20151001-154056/PR07_Bvot_T1T2.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-2002-resultats-par-bureaux-de-vote/20150925-110503/PR02_BVot_T1T2.txt',
 'https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-1er-tour-par-bureaux-de-vote/20170427-100955/PR17_BVot_T1_FE.txt']

In [11]:
reg_ex_test = url_list[0]
reg_ex_test

'https://www.data.gouv.fr/s/resources/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-2nd-tour-par-bureaux-de-vote/20170511-093541/PR17_BVot_T2_FE.txt'

In [12]:
# Here I try to use part of the url as future name for my csv files. I need to keep the nature of the data
reg_ex_test = url_list[0]
url_pattern = 'https(.*)(?=-\d{4}-)'
reg_ex_test = re.sub(url_pattern,'',reg_ex_test)
url_pattern_2 = '\/(.*)'
reg_ex_test = re.sub(url_pattern_2,'',reg_ex_test)
url_pattern_3 = '-(?=\d{4})'
reg_ex_test = re.sub(url_pattern_3,'',reg_ex_test)
reg_ex_test

'2017-resultats-definitifs-du-2nd-tour-par-bureaux-de-vote'

In [13]:
# Here, I will download the files that are behind the url in the list I have created
# inspiration for the code : https://www.kite.com/python/answers/how-to-download-a-csv-file-from-a-url-in-python

for url in url_list:
    req = requests. get(url)
    url_content = req.content
    
    url_pattern = 'https(.*)(?=-\d{4}-)'
    file_name = re.sub(url_pattern,'',url)
    url_pattern_2 = '\/(.*)'
    file_name = re.sub(url_pattern_2,'',file_name)
    url_pattern_3 = '-(?=\d{4})'
    file_name = re.sub(url_pattern_3,'',file_name)
    
    csv_file = open(file_name + '.csv', 'wb')
    csv_file. write(url_content)
    csv_file.close()

PermissionError: [Errno 13] Permission denied: '2017-resultats-definitifs-du-1er-tour-par-bureaux-de-vote.csv'